In [27]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import scipy.io as sc
import numpy as np
from numpy import array
import random
import pandas as pd
import sys
import time
from sklearn import preprocessing

In [2]:
# this function is used to transfer one column label to one hot label
def one_hot(y_):
    # Function to encode output labels from number indexes
    # e.g.: [[5], [0], [3]] --> [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    y_ = y_.reshape(len(y_))
    n_values = np.max(y_) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]

In [35]:
#  Data loading
all_data = pd.read_csv("user1_test10_attention.csv")
feature = array(all_data.iloc[:, :-1])
labels = array(all_data.iloc[:, -1]).reshape(-1, 1)

print (feature.shape, labels.shape)

(2400, 8) (2400, 1)


In [39]:
np.random.shuffle(all)   # mix eeg_all

final=240*10
# all=all[0:final]
feature_all = feature
label=labels.astype("int32")

In [40]:
label.shape

(2400, 1)

In [41]:
#z-score
feature_all=preprocessing.scale(feature_all)
no_fea=feature_all.shape[-1]
label_all=one_hot(label)
print (label_all.shape)

(2400, 6)


In [42]:
n_classes=6
###CNN code,
feature_all=feature_all# the input data of CNN
print ("cnn input feature shape", feature_all.shape)
n_fea=feature_all.shape[-1]
# label_all=one_hot(label_all)


cnn input feature shape (2400, 8)


In [44]:
final=feature.shape[0]
middle_number=int(final*3/4)
feature_training =feature_all[0:middle_number]
feature_testing =feature_all[middle_number:final]
label_training =label_all[0:middle_number]
label_testing =label_all[middle_number:final]
label_ww=label_all[middle_number:final]  # for the confusion matrix
print ("label_testing",label_testing.shape)
a=feature_training
b=feature_testing
print(feature_training.shape)
print(feature_testing.shape)

label_testing (600, 6)
(1800, 8)
(600, 8)


In [45]:
keep=1
batch_size=final-middle_number
n_group=3
train_fea=[]
for i in range(n_group):
    f =a[(0+batch_size*i):(batch_size+batch_size*i)]
    train_fea.append(f)
print (train_fea[0].shape)

(600, 8)


In [46]:
train_label=[]
for i in range(n_group):
    f =label_training[(0+batch_size*i):(batch_size+batch_size*i), :]
    train_label.append(f)
print (train_label[0].shape)


(600, 6)


In [47]:
# the CNN code
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess3.run(prediction, feed_dict={xs: v_xs, keep_prob: keep})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess3.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: keep})
    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    # stride [1, x_movement, y_movement, 1]
    # Must have strides[0] = strides[3] = 1
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# def max_pool_2x2(x):
#     # stride [1, x_movement, y_movement, 1]
#     return tf.nn.max_pool(x, ksize=[1,1,2,1], strides=[1,1,2,1], padding='SAME')
def max_pool_1x2(x):
    # stride [1, x_movement, y_movement, 1]
    return tf.nn.max_pool(x, ksize=[1,1,2,1], strides=[1,1,2,1], padding='SAME')

In [48]:
# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, n_fea]) # 1*64
ys = tf.placeholder(tf.float32, [None, n_classes])  # 2 is the classes of the data
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 1, n_fea, 1])
print(x_image.shape)  # [n_samples, 28,28,1]

## conv1 layer ##
W_conv1 = weight_variable([1,1, 1,20]) # patch 1*1, in size is 1, out size is 2
b_conv1 = bias_variable([20])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) # output size 1*64*2
h_pool1 = max_pool_1x2(h_conv1)                          # output size 1*32x2

## conv2 layer ##
# W_conv2 = weight_variable([1,1, 2, 4]) # patch 1*1, in size 2, out size 4
# b_conv2 = bias_variable([4])
# h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) # output size 1*32*4
# h_pool2 = max_pool_1x2(h_conv2)                          # output size 1*16*4

## fc1 layer ##
W_fc1 = weight_variable([int(1*(n_fea/2)*20), 120])
b_fc1 = bias_variable([120])
# [n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]
h_pool2_flat = tf.reshape(h_pool1, [-1, int(1*(n_fea/2)*20)])
h_fc1 = tf.nn.sigmoid(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## fc2 layer ##
W_fc2 = weight_variable([120, n_classes])
b_fc2 = bias_variable([n_classes])
prediction = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

# the error between prediction and real data
l2 = 0.001 * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=ys))+l2   # Softmax loss
train_step = tf.train.AdamOptimizer(0.04).minimize(cross_entropy) # learning rate is 0.0001

(?, 1, 8, 1)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
sess3 = tf.Session()
init = tf.global_variables_initializer()
sess3.run(init)

np.set_printoptions(threshold=sys.maxsize)
step = 1
while step < 10000:
    for i in range(n_group):
        sess3.run(train_step, feed_dict={xs: train_fea[i], ys: train_label[i], keep_prob:keep})
    if step % 5 == 0:
        cost=sess3.run(cross_entropy, feed_dict={xs: b, ys: label_testing, keep_prob: keep})
        acc_cnn_t=compute_accuracy(b, label_testing)
        print('the step is:',step,',the acc is',acc_cnn_t,', the cost is', cost)
    step+=1
acc_cnn=compute_accuracy(b, label_testing)
time2=time.clock()
feature_all_cnn=sess3.run(h_fc1_drop, feed_dict={xs: feature_all, keep_prob: keep})
print ("the shape of cnn output features",feature_all.shape,label_all.shape)

time3=time.clock()

the step is: 5 ,the acc is 0.0 , the cost is 1.5011863
the step is: 10 ,the acc is 0.07333333 , the cost is 1.3991553
the step is: 15 ,the acc is 0.07333333 , the cost is 1.7831342
the step is: 20 ,the acc is 0.07333333 , the cost is 2.190463
the step is: 25 ,the acc is 0.07333333 , the cost is 3.6142266
the step is: 30 ,the acc is 0.07333333 , the cost is 2.4316583
the step is: 35 ,the acc is 0.07333333 , the cost is 3.2935033
the step is: 40 ,the acc is 0.07333333 , the cost is 3.4810603
the step is: 45 ,the acc is 0.075 , the cost is 3.2573636
the step is: 50 ,the acc is 0.07333333 , the cost is 3.9671507
the step is: 55 ,the acc is 0.07333333 , the cost is 3.932888
the step is: 60 ,the acc is 0.07333333 , the cost is 3.1628594
the step is: 65 ,the acc is 0.08166666 , the cost is 3.096047
the step is: 70 ,the acc is 0.083333336 , the cost is 3.4352942
the step is: 75 ,the acc is 0.083333336 , the cost is 3.6554418
the step is: 80 ,the acc is 0.07333333 , the cost is 5.0386605
the st